In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
import numpy as np
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# Load the data
train_df = pd.read_csv('/content/drive/MyDrive/데이콘/open/train.csv')[['ID', 'timestamp', 'supply(kg)', 'price(원/kg)']]
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])

# Function to determine the season
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Extracting day of week and season
train_df['day_of_week'] = train_df['timestamp'].dt.dayofweek
train_df['season'] = train_df['timestamp'].dt.month.apply(get_season)
def map_season_to_number(season):
    mapping = {'Winter': 0, 'Spring': 1, 'Summer': 2, 'Fall': 3}
    return mapping.get(season, -1)  # 기본값으로 -1을 사용

# 계절 열을 숫자로 변환
train_df['season'] = train_df['season'].apply(map_season_to_number)

# Preparing the dataset for AutoGluon
train_df['item_id'] = train_df.ID.str[0:6]
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))

# Creating and training the predictor
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)

# Fit the model
predictor.fit(data, random_seed=42)

# Prediction
# ...

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

In [ ]:
# Prediction
submission = pd.read_csv('/content/drive/MyDrive/데이콘/open/sample_submission.csv')
pred = predictor.predict(data)  # Make sure to pass the correct dataset here
submission['answer'] = pred.reset_index()['mean']
submission.loc[submission['answer'] < 0.0, 'answer'] = 0.0
submission.to_csv('./dacon_submission.csv', index=False)
submission